<a href="https://colab.research.google.com/github/Uniholder/Specialization-Yandex-MIPT/blob/main/6_Final_projects/2_churn_prediction/week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir data
!wget -O data/orange_small_churn_data.train https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
!wget -O data/orange_small_churn_labels.train https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train
!gdown -O data/ 1rW8ZBa870U3DHzERRfy1q777914JKPtA

--2022-08-17 11:27:19--  https://stepik.org/media/attachments/lesson/90608/_847b6cd041836e8fa914243e83d898fb_orange_small_churn_data.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23722578 (23M) [application/octet-stream]
Saving to: ‘data/orange_small_churn_data.train’

data/orange_small_c 100%[===================>]  22.62M  11.8MB/s    in 1.9s    

2022-08-17 11:27:25 (11.8 MB/s) - ‘data/orange_small_churn_data.train’ saved [23722578/23722578]

--2022-08-17 11:27:25--  https://stepik.org/media/attachments/lesson/90608/_ce5e9e4280a8856218f1886f007de2a4_orange_small_churn_labels.train
Resolving stepik.org (stepik.org)... 13.69.75.239
Connecting to stepik.org (stepik.org)|13.69.75.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117024 (114K) [application/octet-stream]
Saving to: ‘data/orange_small_churn_labels.train’

data/ora

# Входные данные

In [ ]:
import pandas as pd
import os
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

PATH_TO_DATA = 'data'

In [ ]:
df_raw = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_data.train'))
labels = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_labels.train'), header=None, squeeze=True)
df_raw.shape

(40000, 230)

In [ ]:
num_vars = df_raw.columns[:190]
cat_vars = df_raw.columns[190:]
na_vars = df_raw.columns[df_raw.isna().all()]

nna_num_vars = list(set(num_vars) - set(na_vars))
nna_cat_vars = list(set(cat_vars) - set(na_vars))

In [ ]:
df_raw['target'] = labels
df = df_raw.dropna(axis=1, how='all')
df.shape

(40000, 213)

Отделим hold-out dataset

In [ ]:
df_train, df_ho = train_test_split(df, test_size=5000)
df_train.shape, df_ho.shape

((35000, 213), (5000, 213))

In [ ]:
# df_ho.to_csv(os.path.join(PATH_TO_DATA, 'hold_out_dataset.csv'))

Preprocessing

In [ ]:
cols = df_train.drop('target', axis=1).columns

In [ ]:
cat_encoder = ColumnTransformer(
    transformers=[
        ("cat", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), nna_cat_vars)
    ], remainder='passthrough'
)

preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('cat_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

In [ ]:
%%time
df_train_enc = pd.DataFrame(preprocessor.fit_transform(df_train[cols]), columns=cols)

CPU times: user 6.74 s, sys: 751 ms, total: 7.49 s
Wall time: 7.49 s


In [ ]:
df_train_enc

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0.0,0.0,0.0,0.0,0.0,123.0,1.0,0.0,0.0,0.0,...,2652.0,4.0,1004.0,0.0,0.0,1.0,6.0,2.0,8.0,0.0
1,0.0,0.0,0.0,0.0,0.0,271.0,4.0,0.0,0.0,0.0,...,305.0,4.0,2352.0,3.0,0.0,1.0,18.0,3.0,15.0,0.0
2,0.0,0.0,0.0,0.0,0.0,65.0,0.0,0.0,0.0,0.0,...,1465.0,4.0,1889.0,0.0,0.0,2.0,10.0,2.0,8.0,1.0
3,0.0,0.0,0.0,0.0,60.0,0.0,1.0,0.0,0.0,0.0,...,3637.0,4.0,3572.0,0.0,0.0,0.0,0.0,2.0,8.0,0.0
4,0.0,0.0,0.0,0.0,0.0,111.0,1.0,0.0,0.0,0.0,...,2285.0,4.0,1421.0,3.0,0.0,0.0,10.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,0.0,0.0,0.0,0.0,0.0,126.0,1.0,0.0,0.0,0.0,...,260.0,4.0,1610.0,1.0,0.0,1.0,0.0,2.0,2.0,0.0
34996,0.0,0.0,0.0,0.0,0.0,122.0,1.0,0.0,0.0,0.0,...,3010.0,4.0,3546.0,0.0,0.0,0.0,10.0,2.0,8.0,0.0
34997,0.0,0.0,0.0,0.0,0.0,95.0,1.0,0.0,0.0,0.0,...,1807.0,6.0,566.0,0.0,0.0,2.0,13.0,1.0,29.0,0.0
34998,0.0,0.0,0.0,0.0,0.0,124.0,1.0,0.0,0.0,0.0,...,71.0,4.0,734.0,0.0,0.0,0.0,12.0,2.0,8.0,0.0


In [ ]:
df_train_enc['Var229'].value_counts()

0.0    28081
1.0     6866
3.0       27
2.0       26
Name: Var229, dtype: int64

In [ ]:
%%time
df_ho_enc = pd.DataFrame(preprocessor.transform(df_ho[cols]), columns=cols)

CPU times: user 3.04 s, sys: 38.1 ms, total: 3.08 s
Wall time: 3.27 s


# Модель

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit, cross_validate
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score

from catboost import CatBoostClassifier, cv, Pool

In [ ]:
X, y = df_train_enc, df_train.target
X_val, y_val = df_ho_enc, df_ho.target
cv_scheme = StratifiedShuffleSplit(random_state=0, )

## LogisticRegression

In [ ]:
%%time
cv_results = cross_validate(estimator=LogisticRegression(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

CPU times: user 366 ms, sys: 140 ms, total: 506 ms
Wall time: 18.4 s


In [ ]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.53476614, 0.55761854, 0.56420703, 0.55347365, 0.57323837,
        0.56892589, 0.57951567, 0.57544368, 0.5675691 , 0.57288743]),
 0.5647645488417107)

In [ ]:
lr = LogisticRegression(random_state=0)
lr.fit(X, y)
print(roc_auc_score(y_val, lr.predict_proba(X_val)[:, 1]))
print(classification_report(y_val, lr.predict(X_val)))

0.5135658880874645
              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      4629
           1       0.00      0.00      0.00       371

    accuracy                           0.93      5000
   macro avg       0.46      0.50      0.48      5000
weighted avg       0.86      0.93      0.89      5000



### SGDClassifier

In [ ]:
%%time
cv_results = cross_validate(estimator=SGDClassifier(random_state=0, loss='log'), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

CPU times: user 703 ms, sys: 106 ms, total: 809 ms
Wall time: 1min 13s


In [ ]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.51876543, 0.43883986, 0.46424027, 0.56143576, 0.51058879,
        0.4664819 , 0.59929962, 0.57223565, 0.48581642, 0.49131691]),
 0.5109020606804557)

In [ ]:
sgdc = SGDClassifier(random_state=0, loss='log')
sgdc.fit(X, y)
print(roc_auc_score(y_val, sgdc.predict_proba(X_val)[:, 1]))
print(classification_report(y_val, sgdc.predict(X_val)))

0.49977902115981576
              precision    recall  f1-score   support

          -1       0.93      0.99      0.96      4629
           1       0.07      0.01      0.01       371

    accuracy                           0.92      5000
   macro avg       0.50      0.50      0.48      5000
weighted avg       0.86      0.92      0.89      5000



## Linear models без predict_proba

In [ ]:
rc = RidgeClassifier(random_state=0)
rc.fit(X, y)
print(rc.score(X_val, y_val))
print(classification_report(y_val, rc.predict(X_val)))

0.9258
              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      4629
           1       0.00      0.00      0.00       371

    accuracy                           0.93      5000
   macro avg       0.46      0.50      0.48      5000
weighted avg       0.86      0.93      0.89      5000



In [ ]:
np.unique(rc.predict(X_val))

array([-1])

In [ ]:
sgdc_hinge = SGDClassifier(random_state=0)
sgdc_hinge.fit(X, y)
print(sgdc_hinge.score(X_val, y_val))
print(classification_report(y_val, sgdc_hinge.predict(X_val)))

0.9002
              precision    recall  f1-score   support

          -1       0.93      0.97      0.95      4629
           1       0.10      0.05      0.06       371

    accuracy                           0.90      5000
   macro avg       0.52      0.51      0.51      5000
weighted avg       0.87      0.90      0.88      5000



In [ ]:
np.unique(sgdc_hinge.predict(X_val), return_counts=True)

(array([-1,  1]), array([4838,  162]))

## RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(oob_score=True)

In [ ]:
rfc.fit(X, y)

RandomForestClassifier(oob_score=True)

In [ ]:
rfc.oob_score_

0.9255142857142857

In [ ]:
%%time
cv_results = cross_validate(estimator=RandomForestClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

CPU times: user 929 ms, sys: 197 ms, total: 1.13 s
Wall time: 1min 52s


In [ ]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.68615503, 0.64888115, 0.65147851, 0.65385348, 0.69890491,
        0.6616417 , 0.67809948, 0.63893709, 0.63717102, 0.69673839]),
 0.6651860770637715)

In [ ]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X, y)
print(roc_auc_score(y_val, rfc.predict_proba(X_val)[:, 1]))
print(classification_report(y_val, rfc.predict(X_val)))

0.6076507591016205
              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      4629
           1       0.00      0.00      0.00       371

    accuracy                           0.93      5000
   macro avg       0.46      0.50      0.48      5000
weighted avg       0.86      0.93      0.89      5000



## GradientBoostingClassifier

In [ ]:
%%time
cv_results = cross_validate(estimator=GradientBoostingClassifier(random_state=0), X=X, y=y, n_jobs=-1, scoring='roc_auc', cv=cv_scheme)

CPU times: user 2.94 s, sys: 381 ms, total: 3.32 s
Wall time: 7min 25s


In [ ]:
cv_results['test_score'], cv_results['test_score'].mean()

(array([0.74062203, 0.72099437, 0.75275641, 0.71550098, 0.76534069,
        0.72266877, 0.73486111, 0.71619357, 0.7206738 , 0.77232401]),
 0.7361935741904585)

In [ ]:
%%time
gbc = GradientBoostingClassifier(random_state=0)
gbc.fit(X, y)
print(roc_auc_score(y_val, gbc.predict_proba(X_val)[:, 1]))
print(classification_report(y_val, gbc.predict(X_val)))

0.6333803823195965
              precision    recall  f1-score   support

          -1       0.93      0.97      0.95      4629
           1       0.11      0.04      0.06       371

    accuracy                           0.90      5000
   macro avg       0.52      0.51      0.51      5000
weighted avg       0.87      0.90      0.88      5000



## CatBoost

In [ ]:
base_params = {
    'eval_metric': 'AUC:hints=skip_train~false', 
    'verbose': False,
    'loss_function': 'Logloss'
}

In [ ]:
%%time
cv_results = cv(
    params={**base_params},
    pool=Pool(X, y),
    fold_count=5,
    shuffle=True,
    # plot=True,
    type='Classical'
)

Training on fold [0/5]

bestTest = 0.7286707179
bestIteration = 720

Training on fold [1/5]

bestTest = 0.7373131384
bestIteration = 645

Training on fold [2/5]

bestTest = 0.7296767735
bestIteration = 702

Training on fold [3/5]

bestTest = 0.7241633164
bestIteration = 515

Training on fold [4/5]

bestTest = 0.7240637773
bestIteration = 497

CPU times: user 6min 35s, sys: 19.3 s, total: 6min 54s
Wall time: 4min 18s


In [ ]:
cv_results

,iterations,test-AUC-mean,test-AUC-std,train-AUC-mean,train-AUC-std,test-Logloss-mean,test-Logloss-std,train-Logloss-mean,train-Logloss-std
0,0,0.512145,0.010277,0.515849,0.002189,0.661528,0.000092,0.661518,0.000073
1,1,0.552348,0.017032,0.563089,0.017614,0.631977,0.000223,0.631923,0.000237
2,2,0.644128,0.007802,0.656502,0.002526,0.602548,0.000187,0.602489,0.000110
3,3,0.647211,0.008819,0.661261,0.001069,0.577617,0.000145,0.577540,0.000042
4,4,0.650984,0.006798,0.663917,0.001575,0.554591,0.000160,0.554514,0.000061
...,...,...,...,...,...,...,...,...,...
995,995,0.726673,0.006105,0.878823,0.002602,0.240820,0.001232,0.197044,0.000944
996,996,0.726636,0.006101,0.878985,0.002652,0.240832,0.001235,0.196984,0.000972
997,997,0.726591,0.006081,0.879063,0.002555,0.240843,0.001224,0.196944,0.000943
998,998,0.726619,0.006127,0.879152,0.002513,0.240839,0.001231,0.196911,0.000934


In [ ]:
cv_results['test-AUC-mean'].argmax(), cv_results['test-AUC-mean'].max()

(616, 0.728358117870255)

In [ ]:
%%time
cat_boost_cl = CatBoostClassifier(**base_params)
cat_boost_cl.fit(X, y)
print(roc_auc_score(y_val, cat_boost_cl.predict_proba(X_val)[:, 1]))
print(classification_report(y_val, cat_boost_cl.predict(X_val)))

0.7064975931066249
              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      4629
           1       0.29      0.01      0.01       371

    accuracy                           0.93      5000
   macro avg       0.61      0.50      0.49      5000
weighted avg       0.88      0.93      0.89      5000

CPU times: user 1min 29s, sys: 2.78 s, total: 1min 31s
Wall time: 1min


## CatBoost out of the box

In [ ]:
df_train[cols]

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,...,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
8988,NaN,NaN,NaN,NaN,NaN,861.0,7.0,NaN,NaN,NaN,...,gmDv8BP,oslk,HCTwS9i,LM8l689qOp,NaN,kG3k,Aoh3,RAYp,F2FyR07IdsN7I,am7c
12847,NaN,NaN,NaN,NaN,NaN,1897.0,28.0,NaN,NaN,NaN,...,4UxGlow,oslk,catzS2D,jySVZNlOJy,NaN,kG3k,szEZ,ZI9m,VjDE,am7c
28412,NaN,NaN,NaN,NaN,NaN,455.0,0.0,NaN,NaN,NaN,...,OXxv4zN,oslk,VWtE4Qn,LM8l689qOp,NaN,xG3x,Qu4f,RAYp,F2FyR07IdsN7I,mj86
27589,NaN,NaN,NaN,NaN,31485.0,NaN,NaN,NaN,0.0,NaN,...,zLzYH7D,oslk,xb5Ty9x,LM8l689qOp,NaN,NaN,3Cy4,RAYp,F2FyR07IdsN7I,NaN
12556,NaN,NaN,NaN,NaN,NaN,777.0,7.0,NaN,NaN,NaN,...,aD1vOIl,oslk,NnYzgmc,jySVZNlOJy,NaN,NaN,Qu4f,RAYp,55YFVY9,am7c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35470,NaN,NaN,NaN,NaN,NaN,882.0,7.0,NaN,NaN,NaN,...,3s4OZ4E,oslk,QpZO8fW,M_8D,NaN,kG3k,3Cy4,RAYp,55YFVY9,am7c
39476,NaN,NaN,NaN,NaN,NaN,854.0,7.0,NaN,NaN,NaN,...,n2dvEWW,oslk,xInqI2t,LM8l689qOp,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN
29123,NaN,NaN,NaN,NaN,NaN,665.0,7.0,NaN,NaN,NaN,...,UF16siJ,zCkv,APgdzOv,LM8l689qOp,NaN,xG3x,Xa3G,6fzt,xwM2aC7IdeMC0,am7c
8642,NaN,NaN,NaN,NaN,NaN,868.0,NaN,NaN,NaN,NaN,...,1FARoKG,oslk,CnZoIp0,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN


In [ ]:
si = SimpleImputer(strategy='most_frequent')
X_imp = pd.DataFrame(si.fit_transform(df_train[cols]), columns=cols)
X_val_imp = pd.DataFrame(si.transform(df_ho[cols]), columns=cols)

In [ ]:
%%time
cat_boost_cl_oob = CatBoostClassifier(**base_params, cat_features=nna_cat_vars)
cat_boost_cl_oob.fit(X_imp, y)
print(roc_auc_score(y_val, cat_boost_cl_oob.predict_proba(X_val_imp)[:, 1]))
print(classification_report(y_val, cat_boost_cl_oob.predict(X_val_imp)))

0.7556119599920575
              precision    recall  f1-score   support

          -1       0.93      1.00      0.96      4629
           1       0.40      0.01      0.02       371

    accuracy                           0.93      5000
   macro avg       0.66      0.50      0.49      5000
weighted avg       0.89      0.93      0.89      5000

CPU times: user 8min 49s, sys: 29 s, total: 9min 18s
Wall time: 5min 3s


In [ ]:
%%time
cv_results = cv(
    params={**base_params},
    pool=Pool(X_imp, y, cat_features=nna_cat_vars),
    fold_count=5,
    shuffle=True,
    # plot=True,
    type='Classical'
)

Training on fold [0/5]

bestTest = 0.736551783
bestIteration = 673

Training on fold [1/5]

bestTest = 0.7489355096
bestIteration = 857

Training on fold [2/5]

bestTest = 0.7328463226
bestIteration = 701

Training on fold [3/5]

bestTest = 0.7424870962
bestIteration = 871

Training on fold [4/5]

bestTest = 0.7397731753
bestIteration = 497

CPU times: user 40min 24s, sys: 3min 24s, total: 43min 48s
Wall time: 22min 54s
